Loads Network performance data from ES. Learns to predict bandwidth from packet loss and one way delay.

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import re,time
from elasticsearch import Elasticsearch, exceptions as es_exceptions
from elasticsearch.helpers import scan
es = Elasticsearch(hosts=[{'host':'atlas-kibana.mwt2.org', 'port':9200}],timeout=60)



In [2]:
ind='network_weather_2-2016.6.21'

currentTime=int(round(datetime.utcnow().timestamp()* 1000))
lastHours=24
startTime=currentTime - lastHours*3600000 
endTime=currentTime

source='WT2'
destination='AGLT2'

In [3]:
myquery={
#    "size":50,
    "fields": ["timestamp", "delay_mean","throughput","packet_loss"],
    "query":{
        "filtered":{
            "query":{
                "bool" : {
                  "must" :     [ 
                        {"term": {"srcProduction": {"value": True } } },
                        {"term": {"destProduction": {"value": True } } },
                        {"term": {"srcSite": { "value": source } } },
                        {"term": {"destSite": { "value": destination } } }
                    ]
                }
#               "query_string":{"query":queryString,"analyze_wildcard":True,"lowercase_expanded_terms":False}
            }
#            ,"filter":{ "range":{"timestamp":{"gte": startTime,"lte": endTime,"format": "epoch_millis"}}   }
        }
    }
#    ,"sort": { "timestamp": { "order": "asc" }}
}

In [4]:
scroll = scan(es, query=myquery, index=ind, scroll='10s')
RawData={}
count=0
for res in scroll:
    #print(res)
    r = res['fields']
    sts = r['timestamp'][0]
    ts=time.mktime(datetime.strptime(sts, "%Y-%m-%dT%H:%M:%S").timetuple())
    if not ts in RawData:
        RawData[ts]=[-1,-1,-1]
    if res['_type']=='latency':
        RawData[ts][0]=r['delay_mean'][0]
    elif res['_type']=='packet_loss_rate':
        RawData[ts][1]=r['packet_loss'][0]
    elif res['_type']=='throughput':
        RawData[ts][2]=r['throughput'][0]
    count+=1
    
print ("results fetched:", count ) 
print ("results summed up:", len(RawData) ) 

tss = sorted(RawData.keys())
for r in tss:
    print(r, RawData[r])


results fetched: 43
results summed up: 26
1466533800.0 [27.497766666666664, -1, -1]
1466534100.0 [27.295995995995995, -1, -1]
1466534400.0 [27.325617078052037, -1, -1]
1466534700.0 [27.2765, -1, -1]
1466535000.0 [27.03626666666666, -1, -1]
1466535300.0 [27.025175058352794, -1, -1]
1466535600.0 [26.95736666666666, -1, -1]
1466535900.0 [27.73908544726303, -1, -1]
1466536200.0 [27.129166666666666, -1, -1]
1466536500.0 [26.712408272181456, 0.0006666666666666666, -1]
1466536800.0 [26.710636878959658, 0.0003333333333333333, -1]
1466537100.0 [27.03352253756261, 0.0016666666666666668, -1]
1466537400.0 [26.889382303839735, 0.0016666666666666668, -1]
1466537700.0 [26.838333333333335, 0.0, -1]
1466538000.0 [26.844599999999993, 0.0, -1]
1466538300.0 [27.00853617872625, 0.0003333333333333333, -1]
1466538600.0 [27.046446446446446, 0.001, -1]
1466538900.0 [27.0421087754421, 0.001, -1]
1466539200.0 [27.00587253920586, 0.001, -1]
1466539500.0 [26.97476666666667, 0.0, -1]
1466539800.0 [26.98410000000001

cleaning the data - create array where all 3 components are there.

In [5]:
X=[]
y=[]
for (k,v) in RawData.items():
    if v[2]==-1: continue
    delay_measurements=0
    pl_measurements=0
    delay_sum=0
    pl_sum=0
    for (k1,v1) in RawData.items():
        if abs(k1-k)<600:
            if v1[0]>-1:
                delay_measurements+=1
                delay_sum+=v1[0]
            if v1[1]>-1:
                pl_measurements+=1
                pl_sum+=v1[1]
            #print(k, k1)
    if pl_measurements>0 and delay_measurements>0:
#         print (k, v, delay_sum/delay_measurements, pl_sum/pl_measurements)
        X.append([ delay_sum/delay_measurements, pl_sum/pl_measurements ])
        if v[2]>2*1024*1024*8:
            y.append('good')
        else:
            y.append('bad')
print(X)
print(y)

[]
[]


takes as input two arrays: an array X of size [n_samples, n_features] holding the training samples, and an array y of class labels (strings or integers), size [n_samples]

Learning step:

In [6]:
from sklearn import svm
# X = [[0, 0], [1, 1], [1, 1.2], [0.5, 0.8]]
#y = ['good', 'bad']
clf = svm.SVC()
f = clf.fit(X, y)
print(f)


/home/ivukotic/anaconda3/lib/python3.5/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


ValueError: Found array with 0 feature(s) (shape=(1, 0)) while a minimum of 1 is required.

Using what was learned

In [ ]:
clf.predict([[2., 2.]])